In [3]:
%cd  ../utils

/net/cephfs/home/tfang/MLG/task_1/utils


In [4]:
import numpy as np
import torch

import scipy
from scipy import stats
from tqdm import tqdm
import copy
import time
import logging

/data/tfang/conda-envs/py309_MLG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%reload_ext autoreload
%autoreload 2


from data_loader import *
from dataset import HistoneDataset_returngenenames

from modified_DeepHistone_model import DeepHistone
from modified_DeepHistone_utils import model_train,model_eval,model_predict
from modified_DeepHistone_utils import get_reshaped_data
from modified_DeepHistone_utils import get_dict_from_data
from modified_DeepHistone_utils import get_dict_from_data_submisson

In [15]:

model_save_folder="../data/DeepHistone/"


left_flank_size = 1000#500#1000
right_flank_size = left_flank_size#500#1000
seq_bin_size=left_flank_size+right_flank_size
histone_bin_size = 1 #100 ,20 ,5,1

seq_bins=seq_bin_size
assert seq_bin_size % histone_bin_size==0
histone_bins=int(seq_bin_size/histone_bin_size)


use_gpu = torch.cuda.is_available()


batchsize=30#10000 # 20, 30

conv_ksize,tran_ksize=9,4 

use_seq=False



best_model_name="prefix-basic-model--use_seq-False-left_flank_size-1000-histone_bin_size-1-conv_ksize-9-tran_ksize-4-batchsize-30-epochs-30-use_gpu-True-time20220404-103222-epoch18-model-final.txt"




In [5]:
valid_chr=[5, 10, 15, 20]
test_chr=[2,18]
train_chr=[i for i in range(1,23) if (i not in valid_chr+test_chr)]
all_genes = load_train_genes()
all_genes.head(n=3)

,gene_name,chr,gene_start,gene_end,TSS_start,TSS_end,strand,gex,cell_line
0,SLC20A1,2,112645939,112663825,112658362,112658412,+,0.000000,1
1,C11orf58,11,16613132,16758340,16738643,16738693,+,2239.103328,1
2,ZSCAN9,6,28224886,28233487,28225263,28225313,+,19.798064,1


In [6]:
# Get genes
train_genes=filter_genes_by_chr(all_genes,train_chr)
valid_genes=filter_genes_by_chr(all_genes,valid_chr)
test_genes=filter_genes_by_chr(all_genes,test_chr)


n_genes_train, _ = np.shape(train_genes)
n_genes_valid, _ = np.shape(valid_genes)
n_genes_test, _ = np.shape(test_genes)



In [16]:
submission_genes=load_test_genes()
submission_genes['cell_line'] = 3
n_genes_submission, _ = np.shape(submission_genes)
submission_genes.head(n=3)

,gene_name,chr,gene_start,gene_end,TSS_start,TSS_end,strand,cell_line
0,CAPN9,1,230747384,230802003,230791876,230791926,+,3
1,ILF2,1,153661788,153671028,153664424,153664474,-,3
2,ST6GALNAC5,1,76867480,77067546,76867440,76867490,+,3


In [8]:
test_genes.head(n=3)

,gene_name,chr,gene_start,gene_end,TSS_start,TSS_end,strand,gex,cell_line
0,SLC20A1,2,112645939,112663825,112658362,112658412,+,0.000000,1
5,POMC,2,25160853,25168903,25168640,25168690,-,0.480536,1
11,AKAIN1,18,5142911,5197503,5197453,5197503,-,0.000000,1


In [1]:
%%time
# Load train data
train_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(train_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_train)

# Load valid data
valid_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(valid_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_valid)

# Load test data
test_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(test_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_valid)

NameError: name 'torch' is not defined

In [ ]:
%%time
# Load submission data 
submission_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(submission_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_submission)



In [ ]:
%%time 
# Run train loader
x_train_histone,x_train_seq,y_train,train_index=get_reshaped_data(dataloader=train_dataloader)

# Run valid loader
x_valid_histone,x_valid_seq,y_valid,valid_index=get_reshaped_data(dataloader=valid_dataloader)

# Run test loader
x_test_histone,x_test_seq,y_test,test_index=get_reshaped_data(dataloader=test_dataloader)



In [ ]:
%%time 
# Run submission loader
x_submission_histone,x_submission_seq,submission_index=get_reshaped_data(dataloader=submission_dataloader,is_train=False)


In [ ]:
print(len(train_index),len(valid_index),len(test_index),len(submission_index))


In [ ]:
%%time 
dna_dict= get_dict_from_data(train_index,valid_index,test_index,
                             x_train_seq,x_valid_seq,x_test_seq)

histone_dict= get_dict_from_data(train_index,valid_index,test_index,
                             x_train_histone,x_valid_histone,x_test_histone,)
gex_dict = get_dict_from_data(train_index,valid_index,test_index,
                             y_train,y_valid,y_test,)

In [ ]:
submission_dna_dict= get_dict_from_data_submisson(submission_index,x_submission_seq)

submission_histone_dict= get_dict_from_data_submisson(submission_index,x_submission_histone,)



In [ ]:
#print(dna_dict['1_FERMT2'].shape,histone_dict['1_FERMT2'].shape,gex_dict['1_FERMT2'].shape)

In [17]:
model = DeepHistone(use_gpu,use_seq=use_seq,bin_list=[seq_bins,histone_bins],inside_ksize=[conv_ksize,tran_ksize])
#model.load_state_dict(torch.load(f"{model_save_folder}{best_model_name}"))
model.forward_fn.load_state_dict(torch.load(f"{model_save_folder}{best_model_name}",map_location=torch.device('cpu')))

# model.forward_fn.load_state_dict(torch.load(f"{model_save_folder}{best_model_name}",map_location=torch.device('cpu')))
# !!! after this step even if i just pritn something , top -i shows im using more than 100 cpus ? 
# is this because the model is orignally trained on gpu ???

#  12972   

<All keys matched successfully>

In [ ]:
('Begin predicting...')
test_gex,test_pred = model_predict(test_index,model,batchsize,dna_dict,histone_dict,gex_dict,)	
test_score = scipy.stats.spearmanr(test_pred , test_gex ).correlation

print('Spearman Correlation Score: {}'.format(test_score))

In [ ]:
# np.savetxt(f"{model_save_folder}label.txt", valid_gex, fmt='%d', delimiter='\t')

In [ ]:
np.savetxt(f"{model_save_folder}submission_pred.txt",submission_pred, fmt='%.4f', delimiter='\t')

In [11]:
submission_pred=np.loadtxt(f"{model_save_folder}submission_pred.txt",delimiter='\t')



In [23]:
submission_pred.flatten()

array([0.000000e+00, 1.018480e+01, 1.416000e-01, ..., 1.542500e+00,
       2.917000e+00, 1.821601e+02])